# **LP3_Run1**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [ ]:
problem = """The PRODA, S.A. industrial products firm has to face the problem of scheduling
the weekly production of its three products (P1, P2 and P3). These products are
sold to large industrial firms and PRODA, S.A. wishes to supply its products in
quantities that are more profitable for it.

Each product entails three operations contributing to the costs: smelting; mechanisation; assembly and
packaging. The smelting operations for products P1 and P2 could be subcontracted, but the smelting operation for product P3 requires special equipment, thus
preventing the use of subcontracts. PRODA also want to know, how much they should subcontract.

For product P1 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.30$
- subcontracted smelting: 0.50$
- mechanisation: 0.20$
- Assembly and packaging: 0.3$
The unit sales price is 1.50$.

For product P2 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.50$
- subcontracted smelting: 0.60$
- mechanisation: 0.10$
- Assembly and packaging: 0.20$
The unit sales price is 1.80$.

For product P3 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.40$
- mechanisation: 0.27$
- Assembly and packaging: 0.20$
The unit sales price is 1.97$.

Each unit of product P1 requires 6 min of smelting time (if performed at PRODA, S.A.), 6 min of mechanisation time and 3 min of assembly and packaging time, respectively. For product P2, the times are 10, 3 and 2 min, respectively. One unit of product P3 needs 8 min of smelting time, 8 min of mechanisation and 2 min for assembly and packaging. PRODA, S.A. has weekly capacities of 8,000 min of smelting time, 12,000 min of mechanisation time and 10,000 min of assembly and packaging time.
The objective is to maximize weekly profits."""

In [ ]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [ ]:
print(response1.choices[0].message.content)

The variables in this mathematical optimization problem are:

1. X1: Number of units of product P1 to be produced
2. X2: Number of units of product P2 to be produced
3. X3: Number of units of product P3 to be produced
4. X4: Number of units of product P1 to be smelted at PRODA
5. X5: Number of units of product P2 to be smelted at PRODA
6. X6: Number of units of product P1 to be smelted through subcontractors
7. X7: Number of units of product P2 to be smelted through subcontractors.


In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
print(response2.choices[0].message.content)

The objective function to maximize weekly profits is:

Max Z = 1.50*(X1+X6+X4) - 0.30*X4 - 0.50*X6 - 0.2*X1 - 0.3*X1 + 1.80*(X2+X7+X5) - 0.50*X5 - 0.60*X7 - 0.1*X2 - 0.2*X2 + 1.97*X3 - 0.40*X3 - 0.27*X3 - 0.2*X3


In [ ]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
print(response3.choices[0].message.content)

1. X1 >= 0, X2 >= 0, X3 >= 0, X4 >= 0, X5 >= 0, X6 >= 0, X7 >= 0 (Non-negativity constraint)

2. X4 + X6 = X1 (All of product P1 must be either smelted at PRODA or subcontracted)

3. X5 + X7 = X2 (All of product P2 must be either smelted at PRODA or subcontracted)

4. 6*X4 + 10*X5 + 8*X3 <= 8000 (Smelting time constraint)

5. 6*X1 + 3*X2 + 8*X3 <= 12000 (Mechanisation time constraint)

6. 3*X1 + 2*X2 + 2*X3 <= 10000 (Assembly and packaging time constraint)

7. X1 = X4 + X6 (The number of units of product P1 smelted at PRODA and subcontracted equals the total units of product P1 produced)

8. X2 = X5 + X7 (The number of units of product P2 smelted at PRODA and subcontracted equals the total units of product P2 produced)

Note: Penultimate two constraints ensure no overproduction or underproduction.


### **Generate Pyomo Code**

In [ ]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response4.choices[0].message.content)

Here is a Python Optimisation code using the pyomo module:
```python
from pyomo.environ import *

# create a model
model = ConcreteModel()

# declare decision variables
model.X1 = Var(domain=NonNegativeReals)
model.X2 = Var(domain=NonNegativeReals)
model.X3 = Var(domain=NonNegativeReals)
model.X4 = Var(domain=NonNegativeReals)
model.X5 = Var(domain=NonNegativeReals)
model.X6 = Var(domain=NonNegativeReals)
model.X7 = Var(domain=NonNegativeReals)

# define the objective function
model.obj = Objective(expr = 1.50*(model.X1+model.X6+model.X4) - 0.30*model.X4 - 0.50*model.X6 - 0.2*model.X1 - 0.3*model.X1 + 1.80*(model.X2+model.X7+model.X5) - 0.50*model.X5 - 0.60*model.X7 - 0.1*model.X2 - 0.2*model.X2 + 1.97*model.X3 - 0.40*model.X3 - 0.27*model.X3, sense=maximize)

# define the constraints
model.con1 = Constraint(expr = model.X4 + model.X6 == model.X1)
model.con2 = Constraint(expr = model.X5 + model.X7 == model.X2)
model.con3 = Constraint(expr = 6*model.X4 + 10*model.X5 + 8*model.X3 <= 8000)
model.con4 = Constraint(expr = 6*model.X1 + 3*model.X2 + 8*model.X3 <= 12000)
model.con5 = Constraint(expr = 3*model.X1 + 2*model.X2 + 2*model.X3 <= 10000)
model.con6 = Constraint(expr = model.X1 == model.X4 + model.X6)
model.con7 = Constraint(expr = model.X2 == model.X5 + model.X7)

# solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# print the results
print("The optimal solution is:")
print("X1 =", model.X1())
print("X2 =", model.X2())
print("X3 =", model.X3())
print("X4 =", model.X4())
print("X5 =", model.X5())
print("X6 =", model.X6())
print("X7 =", model.X7())
print("The maximum weekly profit is", model.obj())
```


### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# declare decision variables
model.X1 = Var(domain=NonNegativeReals)
model.X2 = Var(domain=NonNegativeReals)
model.X3 = Var(domain=NonNegativeReals)
model.X4 = Var(domain=NonNegativeReals)
model.X5 = Var(domain=NonNegativeReals)
model.X6 = Var(domain=NonNegativeReals)
model.X7 = Var(domain=NonNegativeReals)

# define the objective function
model.obj = Objective(expr = 1.50*(model.X1+model.X6+model.X4) - 0.30*model.X4 - 0.50*model.X6 - 0.2*model.X1 - 0.3*model.X1 + 1.80*(model.X2+model.X7+model.X5) - 0.50*model.X5 - 0.60*model.X7 - 0.1*model.X2 - 0.2*model.X2 + 1.97*model.X3 - 0.40*model.X3 - 0.27*model.X3, sense=maximize)

# define the constraints
model.con1 = Constraint(expr = model.X4 + model.X6 == model.X1)
model.con2 = Constraint(expr = model.X5 + model.X7 == model.X2)
model.con3 = Constraint(expr = 6*model.X4 + 10*model.X5 + 8*model.X3 <= 8000)
model.con4 = Constraint(expr = 6*model.X1 + 3*model.X2 + 8*model.X3 <= 12000)
model.con5 = Constraint(expr = 3*model.X1 + 2*model.X2 + 2*model.X3 <= 10000)
model.con6 = Constraint(expr = model.X1 == model.X4 + model.X6)
model.con7 = Constraint(expr = model.X2 == model.X5 + model.X7)

# solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# print the results
print("The optimal solution is:")
print("X1 =", model.X1())
print("X2 =", model.X2())
print("X3 =", model.X3())
print("X4 =", model.X4())
print("X5 =", model.X5())
print("X6 =", model.X6())
print("X7 =", model.X7())
print("The maximum weekly profit is", model.obj())

The optimal solution is:
X1 = 0.0
X2 = 4000.0
X3 = 0.0
X4 = 0.0
X5 = 800.0
X6 = 0.0
X7 = 3200.0
The maximum weekly profit is 10880.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [ ]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# declare decision variables
model.X1 = Var(domain=NonNegativeReals)
model.X2 = Var(domain=NonNegativeReals)
model.X3 = Var(domain=NonNegativeReals)
model.X4 = Var(domain=NonNegativeReals)
model.X5 = Var(domain=NonNegativeReals)
model.X6 = Var(domain=NonNegativeReals)
model.X7 = Var(domain=NonNegativeReals)

# define the objective function
model.obj = Objective(expr = 1.50*(model.X1+model.X6+model.X4) - 0.30*model.X4 - 0.50*model.X6 - 0.2*model.X1 - 0.3*model.X1 + 1.80*(model.X2+model.X7+model.X5) - 0.50*model.X5 - 0.60*model.X7 - 0.1*model.X2 - 0.2*model.X2 + 1.97*model.X3 - 0.40*model.X3 - 0.27*model.X3 - 0.2*model.X2, sense=maximize)

# define the constraints
model.con1 = Constraint(expr = model.X4 + model.X6 == model.X1)
model.con2 = Constraint(expr = model.X5 + model.X7 == model.X2)
model.con3 = Constraint(expr = 6*model.X4 + 10*model.X5 + 8*model.X3 <= 8000)
model.con4 = Constraint(expr = 6*model.X1 + 3*model.X2 + 8*model.X3 <= 12000)
model.con5 = Constraint(expr = 3*model.X1 + 2*model.X2 + 2*model.X3 <= 10000)
model.con6 = Constraint(expr = model.X1 == model.X4 + model.X6)
model.con7 = Constraint(expr = model.X2 == model.X5 + model.X7)

# solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# print the results
print("The optimal solution is:")
print("X1 =", model.X1())
print("X2 =", model.X2())
print("X3 =", model.X3())
print("X4 =", model.X4())
print("X5 =", model.X5())
print("X6 =", model.X6())
print("X7 =", model.X7())
print("The maximum weekly profit is", model.obj())

The optimal solution is:
X1 = 0.0
X2 = 4000.0
X3 = 0.0
X4 = 0.0
X5 = 800.0
X6 = 0.0
X7 = 3200.0
The maximum weekly profit is 10080.0
